In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%aimport xmodexp

In [4]:
import pandas as pd
from xmodexp import *
import pytorch_lightning as pl
import os

In [5]:
import transformers

In [6]:
from transformers import CLIPModel, CLIPProcessor
import ray
import ray.data
import pyarrow as pa

In [39]:
ray.init('auto')

2021-12-02 00:54:31,931	INFO worker.py:822 -- Connecting to existing Ray cluster at address: 172.31.130.221:6379


{'node_ip_address': '172.31.130.221',
 'raylet_ip_address': '172.31.130.221',
 'redis_address': '172.31.130.221:6379',
 'object_store_address': '/tmp/omoll/raytmp/session_2021-12-02_00-54-23_736528_40016/sockets/plasma_store',
 'raylet_socket_name': '/tmp/omoll/raytmp/session_2021-12-02_00-54-23_736528_40016/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/omoll/raytmp/session_2021-12-02_00-54-23_736528_40016',
 'metrics_export_port': 58816,
 'node_id': '2f9f1931cbea7439de99c7e8328c1ae75dffc783d43a12527c3040e5'}

In [8]:
#ray.available_resources()

In [9]:
# fine-tuning clip:
# 1. batch size and learning rate: learning rates in 1e-4 rate seem to not work (training loss flat)
# 2. learning rates below 1e-5 seem ok.
# 3. for specific datset, validation loss decreases and then  rises again after only a hundred steps (for batch size 128).
 ### what is going on here?
    
    

In [10]:
model = CLIPModel.from_pretrained("./models/clip-vit-base-patch32")
#processor = CLIPProcessor.from_pretrained("./models/clip-vit-base-patch32")

In [11]:
# al = list(model.named_parameters())

In [12]:
#'text_model.encoder.layers.0.layer_norm1.weight'
#'text_model.encoder.layers.0.self_attn.out_proj.bias'

In [13]:
#model.get_parameter('logit_scale')

In [14]:
#model.text_model

In [15]:
def has_layer_norm(p):
    return p.find('layer_norm') >= 0 or p.find('layernorm') >= 0

In [16]:
def projection(p):
    return p.find('visual_projection')

In [17]:
def exclude_decay(p):
    return p.endswith('bias') or p.find(layer_norm) >= 0

# [ p for (n,p) in model.named_parameters() if name_pred(n)]

In [18]:
# monitor: 'logit_scale'
# exclude endswith('bias')
# exclude find('layer')

In [19]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.cloud_io import load as pl_load
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback

In [20]:
if False:
    bird_df = pd.read_parquet('./data/bird_guide_df.parquet')

    #bird_df = bird_df.assign(rel_path=bird_df.image_url.map(lambda s : s.replace('https://www.allaboutbirds.org/','') if s is not None else None))
    abs_paths = '/home/gridsan/omoll/data/bird_guide/images/' + bird_df.rel_path
    valid_paths = abs_paths[bird_df.path_ok]
    ds = ray.data.read_binary_files(valid_paths.values.tolist())
    ds.write_parquet('/home/gridsan/omoll/data/bird_guide/images_parquet')
    tab = pa.parquet.read_table('/home/gridsan/omoll/data/bird_guide/images_parquet')
    df = tab.to_pandas()

    df2 = df.rename({0:'image_bytes'}, axis=1).set_index(valid_paths.index)
    bdf3 = bird_df.assign(image_bytes=df2.image_bytes)
    rd2 = ray.data.from_pandas(bdf3)
    rd2.repartition(num_blocks=30).write_parquet('/home/gridsan/omoll/data/bird_guide_single_parquet')

In [21]:
import datasets
from datasets import Dataset
import torch

In [22]:
import pytorch_lightning as pl
import pyarrow as pa

In [72]:
import math

def warmup(warmup_epochs, k=1.):
    def fun(epoch):
        if epoch < warmup_epochs:
            return epoch/warmup_epochs #0
        
        curr_epoch = max(epoch - warmup_epochs,1) # >= 1
        return 1./math.sqrt(max(curr_epoch/k, 1.))
    
    return fun

class CLIPModule(pl.LightningModule):
    def __init__(self, path_or_model_name, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        model = CLIPModel.from_pretrained(path_or_model_name)
        model.logit_scale = nn.Parameter(torch.tensor(math.log(self.hparams.init_scale), 
                                  device=model.logit_scale.device,
                                  dtype=model.logit_scale.dtype))
        
        self.model = model

        
    def forward(self, inputs):
        return  self.model(**inputs, return_loss=True)

    def basic_step(self, batch, train_or_val):
        batch_out = self.forward(batch)
        loss = batch_out.loss
        self.log(f'{train_or_val}_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        return self.basic_step(batch, 'val')
    
    def training_step(self, batch, batch_idx):
        self.log('logit_scale', self.model.logit_scale)
        ans =  self.basic_step(batch, 'train')
        return ans
    
    def configure_optimizers(self):
        m = self.model
        h = self.hparams
        
        
        optimizer = transformers.AdamW([ {'name':'logit_scale',
                                         'params':[m.logit_scale], 
                                         'lr':h.logit_scale_lr, 
                                         'weight_decay':h.logit_scale_decay}
                                       ],
                                       lr=h.lr, weight_decay=h.decay)
        
        names = ['text_model','text_projection','vision_model','visual_projection']
        
        for name in names:
            lr = h[name + '_lr']
            decay = h[name + '_decay']

            submod = m.get_submodule(name)
            
            bias_params = []
            non_bias_params = []
            for (nm, p) in submod.named_parameters():
                if nm.endswith('bias'):
                    bias_params.append(p)
                else:
                    non_bias_params.append(p)
                    
            
            group_no_bias = {'name':name, 'params':non_bias_params, 'lr':lr, 'weight_decay':decay}
            group_bias = {'name':name +'_biases', 'params':bias_params, 'lr':lr, 'weight_decay':0.}
            optimizer.add_param_group(group_no_bias)
            optimizer.add_param_group(group_bias)

        lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=warmup(h.warmup_epochs))
        return {'optimizer':optimizer, 'lr_scheduler':lr_scheduler}

In [25]:
# want to tune: 
# training batch size.
# optimizer settings: learning rate, learning rates for different groups.
# augmentations on images / descriptions.
## what is that tau doing? does it change over time?

In [26]:
import os
from pytorch_lightning.callbacks import LearningRateMonitor


In [45]:
def clip_fine_tune(config, num_epochs, num_gpus):
    del os.environ["SLURM_NTASKS"]
    del os.environ["SLURM_JOB_NAME"]
    
    tmpdir = os.environ['TMPDIR'] + '/base'
    
    bird_tab = pa.parquet.read_table(f'{tmpdir}/data/bird_guide_single_parquet', 
                                     columns=['description', 'image_bytes'])
    bird_dataset = Dataset(bird_tab).filter(lambda tup : tup['description'] 
                                            is not None and tup['image_bytes'] is not None)
    bird_dataset = bird_dataset.rename_column('description', 'text')
    
    processor = CLIPProcessor.from_pretrained(f'{tmpdir}/xmodexp/notebooks/models/clip-vit-base-patch32')
    
    data_mod = MultiModalDataModule(dataset=bird_dataset, processor=processor,
                           test_size=config['test_size'], batch_size=config['batch_size'],
                                    val_batch_size=config['val_batch_size'],
                                    num_workers=config['num_workers'])
    
    model = CLIPModule(f'{tmpdir}/xmodexp/notebooks/models/clip-vit-base-patch32', **config)

    trainer = pl.Trainer(
        num_sanity_val_steps=0,
        max_epochs=num_epochs,
        gpus=math.ceil(num_gpus),
        logger=TensorBoardLogger(save_dir=tune.get_trial_dir(), name="", version="."),
        progress_bar_refresh_rate=0,
        log_every_n_steps=1,
        callbacks=[
            LearningRateMonitor(logging_interval='epoch'),
            TuneReportCallback(
                {
                    "loss": "val_loss",
#                  "mean_accuracy": "ptl/val_accuracy"
                },
                on="validation_end")
        ])
    
    trainer.validate(model, data_mod)
    trainer.fit(model, data_mod)

In [56]:
def maybe_sample(v):
    if getattr(v, "sample", None) != None:
        return v.sample()
    else:
        return v
sample_config = {k:maybe_sample(v) for (k,v) in config.items()}

In [57]:
CLIPModule(sample_config

{'batch_size': 200,
 'visual_projection_lr': 9.884711724038756e-05,
 'text_projection_lr': 1.020482033673083e-06,
 'vision_model_lr': 0.0035931511812772707,
 'text_model_lr': 7.185947244906206e-06,
 'logit_scale_lr': 7.578311819957885e-06,
 'lr': 0.00012534608148709932,
 'visual_projection_decay': 0.00014136144397295186,
 'text_projection_decay': 3.976322726890028e-05,
 'vision_model_decay': 5.333361532386975e-05,
 'text_model_decay': 0.01675657114007378,
 'logit_scale_decay': 0.030062552966386442,
 'decay': 0.0057545241982642934,
 'init_scale': 5.5059978823237925,
 'warmup_epochs': 0,
 'num_workers': 8,
 'test_size': 1000,
 'val_batch_size': 500}

In [ ]:
model.get_submodule()

In [80]:
lr_scale = tune.loguniform(1e-7, 1e-2)
decay_scale = tune.loguniform(1e-5, 1e-1)

config = {
    'batch_size': tune.choice([64, 128, 200]),

    'visual_projection_lr':lr_scale,
    'text_projection_lr':lr_scale,
    'vision_model_lr':lr_scale,
    'text_model_lr':lr_scale,
    'logit_scale_lr':lr_scale,
    'lr': lr_scale,
    
    'visual_projection_decay':decay_scale,
    'text_projection_decay':decay_scale,
    'vision_model_decay':decay_scale,
    'text_model_decay':decay_scale,
    'logit_scale_decay':decay_scale,
    'decay':decay_scale,

    'init_scale':tune.loguniform(.1, 200.),

    'warmup_epochs':tune.choice([0,1,2,4,8]),
    'num_workers':8,
    'test_size': 1000,
    'val_batch_size':500,
}

In [73]:
cm = CLIPModule("/home/gridsan/omoll/xmodexp/notebooks/models/clip-vit-base-patch32", **sample_config)

In [74]:
cm.configure_optimizers()

{'optimizer': AdamW (
 Parameter Group 0
     betas: (0.9, 0.999)
     correct_bias: True
     eps: 1e-06
     initial_lr: 7.578311819957885e-06
     lr: 7.578311819957885e-06
     name: logit_scale
     weight_decay: 0.030062552966386442
 
 Parameter Group 1
     betas: (0.9, 0.999)
     correct_bias: True
     eps: 1e-06
     initial_lr: 7.185947244906206e-06
     lr: 7.185947244906206e-06
     name: text_model
     weight_decay: 0.01675657114007378
 
 Parameter Group 2
     betas: (0.9, 0.999)
     correct_bias: True
     eps: 1e-06
     initial_lr: 7.185947244906206e-06
     lr: 7.185947244906206e-06
     name: text_model_biases
     weight_decay: 0.0
 
 Parameter Group 3
     betas: (0.9, 0.999)
     correct_bias: True
     eps: 1e-06
     initial_lr: 1.020482033673083e-06
     lr: 1.020482033673083e-06
     name: text_projection
     weight_decay: 3.976322726890028e-05
 
 Parameter Group 4
     betas: (0.9, 0.999)
     correct_bias: True
     eps: 1e-06
     initial_lr: 1.0204820

In [75]:
def tune_clip_asha(name, config, num_samples=10, num_epochs=10, grace_period=5, cpus_per_trial=8, gpus_per_trial=1):
    scheduler = ASHAScheduler(
        max_t=max(grace_period, num_epochs),
        grace_period=grace_period,
        reduction_factor=2)

    reporter = CLIReporter(
        parameter_columns=["batch_size", "init_scale", "vision_model_lr", 'text_model_lr', 'logit_scale_lr'],
        metric_columns=["loss", "training_iteration",],
        max_report_frequency=60,
    )

    analysis = tune.run(
        tune.with_parameters(
            clip_fine_tune,
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial),
        resources_per_trial={
            "cpu": cpus_per_trial,
            "gpu": gpus_per_trial
        },
        metric="loss",
        mode="min",
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        name=name,
        keep_checkpoints_num=2, 
        checkpoint_score_attr="min-loss"
    )

    print("Best hyperparameters found were: ", analysis.best_config)

In [79]:
ray.available_resources()

{'memory': 3961707844614.0,
 'GPU': 15.0,
 'CPU': 1200.0,
 'accelerator_type:V100': 15.0,
 'node:172.31.130.221': 1.0,
 'object_store_memory': 1702160504706.0,
 'node:172.31.130.53': 2.0,
 'node:172.31.130.166': 1.0,
 'node:172.31.130.171': 1.0,
 'node:172.31.130.204': 1.0,
 'node:172.31.130.150': 1.0,
 'node:172.31.130.14': 1.0,
 'node:172.31.130.209': 1.0,
 'node:172.31.130.160': 1.0,
 'node:172.31.130.99': 1.0,
 'node:172.31.130.220': 1.0,
 'node:172.31.130.15': 1.0,
 'node:172.31.130.98': 1.0,
 'node:172.31.130.176': 1.0}

In [81]:
tune_clip_asha(name='exp3', config=config, num_samples=160, num_epochs=40, grace_period=10, gpus_per_trial=1)

2021-12-02 01:49:27,520	WARNING trial_runner.py:246 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (1320 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.


== Status ==
Current time: 2021-12-02 01:49:28 (running for 00:00:00.69)
Memory usage on this node: 13.7/377.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Resources requested: 0/1200 CPUs, 0/15 GPUs, 0.0/3689.63 GiB heap, 0.0/1585.26 GiB objects (0.0/15.0 accelerator_type:V100)
Result logdir: /home/gridsan/omoll/ray_results/exp3
Number of trials: 160/160 (160 PENDING)
+----------------------------+----------+-------+--------------+--------------+-------------------+-----------------+------------------+
| Trial name                 | status   | loc   |   batch_size |   init_scale |   vision_model_lr |   text_model_lr |   logit_scale_lr |
|----------------------------+----------+-------+--------------+--------------+-------------------+-----------------+------------------|
| clip_fine_tune_fe3be_00000 | PENDING  |       |          128 |     3.73131  |       1.27368e-05 |     3.7896e-05  |      7.76796e-07 |
| clip_fine_tune_f

(pid=57957) 2021-12-02 01:49:35.760282: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


(scheduler +1h10s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +1h10s) Warning: The following resource request cannot be scheduled right now: {'CPU': 8.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 64.13ba/s]
(pid=11426, ip=172.31.130.99) 2021-12-02 01:49:40.081769: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=15320, ip=172.31.130.150) 2021-12-02 01:49:40.097490: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=66559, ip=172.31.130.53) 2021-12-02 01:49:40.108658: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=77441, ip=172.31.130.98) 2021-12-02 01:49:40.099420: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=66559, ip=172.31.130.53) 2021-12-02 01:49:40.108658: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudar

...

== Status ==
Current time: 2021-12-02 04:40:49 (running for 02:51:21.59)
Memory usage on this node: 18.7/377.6 GiB
Using AsyncHyperBand: num_stopped=158
Bracket: Iter 40.000: None | Iter 20.000: -4.3741888999938965 | Iter 10.000: -5.878180027008057
Resources requested: 16.0/1200 CPUs, 2.0/15 GPUs, 0.0/3689.63 GiB heap, 0.0/1585.26 GiB objects (0.0/15.0 accelerator_type:V100)
Current best trial: fe3be_00120 with loss=3.216007709503174 and parameters={'batch_size': 128, 'init_scale': 17.08693539204199, 'vision_model_lr': 5.0388709172642817e-05, 'text_model_lr': 3.6322830104757226e-05, 'logit_scale_lr': 0.00022781599914833148}
Result logdir: /home/gridsan/omoll/ray_results/exp3
Number of trials: 160/160 (2 RUNNING, 158 TERMINATED)
+----------------------------+------------+----------------------+--------------+--------------+-------------------+-----------------+------------------+---------+----------------------+
| Trial name                 | status     | loc                  |   batch_

Result for clip_fine_tune_fe3be_00158:
  date: 2021-12-02_04-41-59
  done: false
  experiment_id: 6e9f4c10867a40c3887442f4d65f1252
  hostname: d-10-2-1
  iterations_since_restore: 36
  loss: 3.553079843521118
  node_ip: 172.31.130.98
  pid: 54743
  time_since_restore: 1434.784819841385
  time_this_iter_s: 37.44373869895935
  time_total_s: 1434.784819841385
  timestamp: 1638438119
  timesteps_since_restore: 0
  training_iteration: 36
  trial_id: fe3be_00158
  
Result for clip_fine_tune_fe3be_00150:
  date: 2021-12-02_04-42-05
  done: false
  experiment_id: 64d07d9f5aaf48b39262d6dda811cc9d
  hostname: d-14-15-2
  iterations_since_restore: 39
  loss: 3.399939775466919
  node_ip: 172.31.130.221
  pid: 70616
  time_since_restore: 1770.8591485023499
  time_this_iter_s: 47.207669258117676
  time_total_s: 1770.8591485023499
  timestamp: 1638438125
  timesteps_since_restore: 0
  training_iteration: 39
  trial_id: fe3be_00150
  
Result for clip_fine_tune_fe3be_00158:
  date: 2021-12-02_04-42-37


Result for clip_fine_tune_fe3be_00158:
  date: 2021-12-02_04-43-53
  done: false
  experiment_id: 6e9f4c10867a40c3887442f4d65f1252
  hostname: d-10-2-1
  iterations_since_restore: 39
  loss: 3.600660562515259
  node_ip: 172.31.130.98
  pid: 54743
  time_since_restore: 1548.7333235740662
  time_this_iter_s: 35.9163978099823
  time_total_s: 1548.7333235740662
  timestamp: 1638438233
  timesteps_since_restore: 0
  training_iteration: 39
  trial_id: fe3be_00158
  
Result for clip_fine_tune_fe3be_00158:
  date: 2021-12-02_04-44-40
  done: true
  experiment_id: 6e9f4c10867a40c3887442f4d65f1252
  hostname: d-10-2-1
  iterations_since_restore: 40
  loss: 3.663041114807129
  node_ip: 172.31.130.98
  pid: 54743
  time_since_restore: 1595.4570081233978
  time_this_iter_s: 46.723684549331665
  time_total_s: 1595.4570081233978
  timestamp: 1638438280
  timesteps_since_restore: 0
  training_iteration: 40
  trial_id: fe3be_00158
  
== Status ==
Current time: 2021-12-02 04:44:40 (running for 02:55:12.

2021-12-02 04:44:40,292	INFO tune.py:630 -- Total run time: 10512.79 seconds (10512.52 seconds for the tuning loop).


Best hyperparameters found were:  {'batch_size': 128, 'visual_projection_lr': 0.005393393095168505, 'text_projection_lr': 0.00014453864858663312, 'vision_model_lr': 5.0388709172642817e-05, 'text_model_lr': 3.6322830104757226e-05, 'logit_scale_lr': 0.00022781599914833148, 'lr': 2.5723555839546368e-05, 'visual_projection_decay': 0.08149420777631101, 'text_projection_decay': 0.022803100867131183, 'vision_model_decay': 5.6197902543582653e-05, 'text_model_decay': 0.01651777070548456, 'logit_scale_decay': 0.00018720038359285344, 'decay': 0.008557100269919979, 'init_scale': 17.08693539204199, 'warmup_epochs': 8, 'num_workers': 8, 'test_size': 1000, 'val_batch_size': 500}


In [28]:
#from torch.optim.lr_scheduler import 

    

#torch.optim.lr_scheduler.CosineAnnealingLR()
#torch.optim.lr_scheduler.CosineAnnealingLR()

In [1]:
from ray.tune import Analysis
analysis = Analysis("/home/gridsan/omoll/ray_results/tune_clip_birds5")

In [4]:
trial_dfs = analysis.fetch_trial_dataframes()

In [9]:
analysis.get_best_config(metric='loss', mode='min')

{'batch_size': 64,
 'initial_lr': 4.113375502016347e-06,
 'layer_norm_decay': 3.593129931049946e-05,
 'layer_norm_lr': 0.00013614119775845268,
 'logit_scale_decay': 0.05520739189104816,
 'logit_scale_lr': 1.2211821734964874e-05,
 'num_workers': 8,
 'projection_decay': 3.777279806890133e-05,
 'projection_lr': 9.782652934102177e-06,
 'test_size': 1000,
 'text_embedding_lr': 1.3898176110510575e-06,
 'val_batch_size': 500,
 'vision_embedding_lr': 9.942864263421836e-08,
 'weight_decay': 0.07661199540062676}

In [10]:
trial_dfs

154

In [49]:
analysis.dataframe().sort_values('loss')[['loss', 'training_iteration', 'timesteps_total', 'config/initial_lr', 'config/batch_size']]

,loss,training_iteration,timesteps_total,config/initial_lr,config/batch_size
6,2.848996,2,NaN,0.000012,256
9,2.978055,2,NaN,0.000032,256
18,3.575615,4,NaN,0.000004,128
5,3.595278,4,NaN,0.000003,32
2,3.617851,2,NaN,0.000014,256
14,3.744919,2,NaN,0.000001,32
10,3.926135,1,NaN,0.000002,256
11,4.048364,1,NaN,0.000002,128
8,4.196488,20,NaN,0.000003,64
12,4.252052,1,NaN,0.000027,32


In [45]:
#/home/gridsan/omoll/ray_results/tune_clip_birds/clip_fine_tune_eb2b2_00009_9_batch_size\=64,initial_lr\=0.00017238_2021-11-28_12-31-03/

In [46]:
%load_ext tensorboard

In [47]:
%tensorboard --logdir /home/gridsan/omoll/ray_results/tune_clip_birds/

In [54]:
data_mod = MultiModalDataModule(dataset=bird_dataset, processor=processor, 
                           test_size=1000, batch_size=64, val_batch_size=256, num_workers=10)
model = CLIPModule('./models/clip-vit-base-patch32', batch_size=256, initial_lr=.00001)
trainer = pl.Trainer(gpus=[0], logger=pl.loggers.TensorBoardLogger(save_dir=logdir))
#trainer.fit(model, data_mod)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [55]:
trainer.validate(model, data_mod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-fb105b6e-bbd6-50db-1ecd-33328522f94c,GPU-7dc74432-8f8c-4356-7dd3-c0300ed17ec8]


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 4.088575839996338}
--------------------------------------------------------------------------------


[{'val_loss': 4.088575839996338}]

In [52]:
bird_tab = pa.parquet.read_table('/home/gridsan/omoll/data/bird_guide_single_parquet', 
                             columns=['description', 'image_bytes'])
bird_dataset = Dataset(bird_tab).filter(lambda tup : tup['description'] 
                                    is not None and tup['image_bytes'] is not None)
bird_dataset = bird_dataset.rename_column('description', 'text')

processor = CLIPProcessor.from_pretrained("/home/gridsan/omoll/xmodexp/notebooks/models/clip-vit-base-patch32")

data_mod = MultiModalDataModule(dataset=bird_dataset, processor=processor,
                   test_size=1000, batch_size=256,
                            val_batch_size=500,
                            num_workers=8)


  0%|          | 0/7 [00:00<?, ?ba/s]